### Importar librerias

In [ ]:
!pip install --upgrade spacy
!python -m spacy download es_core_news_lg
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 5.8 MB 8.6 MB/s 
     |████████████████████████████████| 456 kB 51.3 MB/s 
     |████████████████████████████████| 10.1 MB 15 kB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 623 kB 47.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


     |████████████████████████████████| 568.4 MB 13 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
     |████████████████████████████████| 13.7 MB 119 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import spacy
import re 

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


### Lectura de datos

In [ ]:
datasetMaterial = pd.read_json(r'/content/drive/MyDrive/Proyecto1PLN/jsonDatosAjustado.json', 'utf-8')
#datasetMaterial = pd.read_json('/content/drive/MyDrive/Proyecto1PLN/ProyectoPLN.ipynb', encoding='latin-1')


#### Registros unicos

In [ ]:
datasetMaterial['dropkey'] = datasetMaterial['text'] + datasetMaterial['key'] + datasetMaterial['url']

In [ ]:
datasetMaterial= datasetMaterial.drop_duplicates(subset=['url'], keep='first').reset_index(drop=True)

In [ ]:
datasetMaterial.shape

(47571, 10)

#### Imputacion de faltante 

In [ ]:
datasetMaterial.text[datasetMaterial['text']=='']=datasetMaterial.title[datasetMaterial['text']=='']

#### Quitar accentos

In [ ]:
import unicodedata
def strip_accents(s):
   # Retorna el string normalizado si la categoría es Mn es decir https://www.compart.com/en/unicode/category/Mn
   return ''.join(str(c).lower() for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
datasetMaterial['text'] = datasetMaterial['text'].apply(strip_accents)

In [ ]:
datasetMaterial.topic.value_counts(normalize=True)

deportes      0.311618
cultura       0.232768
vida          0.142839
politica      0.133043
economia      0.120872
tecnosfera    0.058859
Name: topic, dtype: float64

### NLP

In [ ]:
nlpSpa = spacy.load('es_core_news_lg')

OSError: ignored

In [ ]:
df=datasetMaterial.sample(frac=0.05)
df

,_id,topic,text,title,year,month,day,key,url,dropkey
42285,{'$oid': '611d9fe8049812a18b175e6e'},vida,el informe planeta vivo 2020 de la wwf y la so...,La vacuna para cuidar la biodiversidad sí exis...,2020,9,15,vida-medio-ambiente-la-vacuna-para-cuidar-la-b...,http://www.eltiempo.com//vida/medio-ambiente/l...,El Informe Planeta Vivo 2020 de la WWF y la So...
10501,{'$oid': '611d9fe3049812a18b1464a5'},cultura,varios dias llevaba el actor colombiano victor...,Apareció en Ecuador el padre del actor colombi...,2020,4,1,cultura-gente-actor-victor-hugo-cabrera-encont...,http://www.eltiempo.com//cultura/gente/actor-v...,Varios días llevaba el actor colombiano Víctor...
6410,{'$oid': '611d9fe2049812a18b141b05'},cultura,la cinta the trial of the chicago 7 (el juicio...,'El Juicio de los 7 de Chicago' triunfa en los...,2021,4,5,cultura-cine-y-tv-el-juicio-de-los-7-de-chicag...,http://www.eltiempo.com//cultura/cine-y-tv/el-...,La cinta The Trial of the Chicago 7 (El Juicio...
34026,{'$oid': '611d9fe6049812a18b162a2c'},politica,el presidente ivan duque respondio este doming...,Coronavirus: Duque explica la estrategia para ...,2020,8,10,politica-gobierno-coronavirus-cual-es-la-estra...,http://www.eltiempo.com//politica/gobierno/cor...,El presidente Iván Duque respondió este doming...
9999,{'$oid': '611d9fe3049812a18b145cfc'},cultura,"un joven, de 18 anos, se convertira en el astr...",Del bachillerato a las estrellas: joven viajar...,2021,7,15,cultura-gente-joven-de-18-anos-viajara-al-espa...,http://www.eltiempo.com//cultura/gente/joven-d...,"Un joven, de 18 años, se convertirá en el astr..."
...,...,...,...,...,...,...,...,...,...,...
45959,{'$oid': '611d9fe9049812a18b17d0e0'},vida,muchas personas aseguran que estan experimenta...,Dos meditaciones para calmar la ansiedad y con...,2020,4,16,vida-meditaciones-cortas-para-relajarse-y-cone...,http://www.eltiempo.com//vida/meditaciones-cor...,Muchas personas aseguran que están experimenta...
28758,{'$oid': '611d9fe4049812a18b15ad56'},economia,"este martes, a las 08:17 a. m., despego del ae...",Con 35 pasajeros se hizo primer vuelo piloto p...,2020,7,21,economia-sectores-en-el-primer-vuelo-piloto-pa...,http://www.eltiempo.com//economia/sectores/en-...,"Este martes, a las 08:17 a. m., despegó del ae..."
30250,{'$oid': '611d9fe5049812a18b15d2cc'},economia,los emprendedores del pais si disponen de fuen...,¿Cómo pueden acceder a financiamiento los empr...,2020,2,28,economia-sectores-consejos-para-que-aprenda-co...,http://www.eltiempo.com//economia/sectores/con...,Los emprendedores del país sí disponen de fuen...
32908,{'$oid': '611d9fe6049812a18b1611fb'},politica,el gobierno nacional les solicito a la comisio...,Gobierno aclara que el que debe retirar la ref...,2021,5,4,politica-congreso-tributaria-gobierno-dice-que...,http://www.eltiempo.com//politica/congreso/tri...,El Gobierno Nacional les solicitó a la comisio...


In [ ]:
df['text'] = df['text'].apply(nlpSpa)

NameError: ignored

In [ ]:
def PosFeatures(doc):

    lst=[token.tag_ for token in doc]
    Adjs=lst.count('ADJ')
    Advs=lst.count('ADV')
    Aux=lst.count('AUX')
    Conj=lst.count('CONJ')
    Det=lst.count('DET')
    Inter=lst.count('INTJ')
    Nouns=lst.count('NOUN')
    Num=lst.count('NUM')
    Part=lst.count('PART')
    Pronoun=lst.count('PRON')
    Punct=lst.count('PUNCT')
    Verb=lst.count('VERB')
    X=[Adjs,Advs,Aux,Conj,Det,Inter,Nouns,Num,Part,Pronoun,Punct,Verb]
    return X


In [ ]:
df['POS']=df['text'].apply(lambda x:PosFeatures(x))
df.loc[:, 'Adjs'] = df.POS.map(lambda x: x[0])
df.loc[:, 'Advs'] = df.POS.map(lambda x: x[1])
df.loc[:, 'Aux'] = df.POS.map(lambda x: x[2])
df.loc[:, 'Conj'] = df.POS.map(lambda x: x[3])
df.loc[:, 'Det'] = df.POS.map(lambda x: x[4])
df.loc[:, 'Inter'] = df.POS.map(lambda x: x[5])
df.loc[:, 'Noun'] = df.POS.map(lambda x: x[6])
df.loc[:, 'Numeric'] = df.POS.map(lambda x: x[7])
df.loc[:, 'Part'] = df.POS.map(lambda x: x[8])
df.loc[:, 'Pronoun'] = df.POS.map(lambda x: x[9])
df.loc[:, 'Punct'] = df.POS.map(lambda x: x[10])
df.loc[:, 'Verb'] = df.POS.map(lambda x: x[11])



In [ ]:
def Lemmas(s):
   return ' '.join(str(c.lemma_) for c in s)



In [ ]:
df['textlemma']=df['text'].apply(lambda x: Lemmas(x))

In [ ]:
df

In [ ]:
df_final=df[['dropkey','Adjs', 'Advs', 'Aux', 'Conj', 'Det', 'Inter', 'Noun',
       'Numeric', 'Part', 'Pronoun', 'Punct', 'Verb', 'textlemma']]

In [ ]:
#df_final.to_csv('/content/drive/MyDrive/Proyecto1PLN/POS_60K_PARTE6.csv')

### Union Dfs

In [ ]:
df_p1=pd.read_csv(r'/content/drive/MyDrive/Analitica/Proyecto1PLN/POS_10K_PARTE1.csv', 'utf-8',delimiter=',',index_col=0)
df_p2=pd.read_csv(r'/content/drive/MyDrive/Analitica/Proyecto1PLN/POS_20K_PARTE2.csv', 'utf-8',delimiter=',',index_col=0)
df_p3=pd.read_csv(r'/content/drive/MyDrive/Analitica/Proyecto1PLN/POS_30K_PARTE3.csv', 'utf-8',delimiter=',',index_col=0)
df_p4=pd.read_csv(r'/content/drive/MyDrive/Analitica/Proyecto1PLN/POS_40K_PARTE4.csv', 'utf-8',delimiter=',',index_col=0)
df_p5=pd.read_csv(r'/content/drive/MyDrive/Analitica/Proyecto1PLN/POS_50K_PARTE5.csv', 'utf-8',delimiter=',',index_col=0)

In [ ]:
df_union=pd.concat([df_p1,df_p2,df_p3,df_p4,df_p5])
df_union

,dropkey,Adjs,Advs,Aux,Conj,Det,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma
0,cultura-musica-y-libros-vallenatos-de-el-trio-...,1,0,0,0,3,0,3,0,0,0,0,0,el vallenato con guitarra de el hijo de ' el c...
1,cultura-gastronomia-entrevista-con-consol-rodr...,1,0,0,0,3,0,2,0,0,0,0,0,el secreto de uno guru de el comida crudivegán
2,OBJ24062586_13 by Yhonatan Loaiza on ScribdEl...,31,8,13,0,42,0,84,6,1,15,67,21,obj24062586_13 by yhonatan loaiza on scribde...
3,"""'Origen' me regresó a la infancia para record...",7,6,6,0,29,0,42,0,0,12,25,19,""" ' origen ' yo regreso a el infancia para rec..."
4,"""A Slow Fire Burning"" es el título de la nueva...",21,2,9,0,41,0,62,4,0,18,49,41,""" a slow fire burning "" ser el titulo de el nu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47566,“Él no es así; él nunca se comportó de esa man...,93,39,53,0,169,0,257,10,0,73,152,140,“ el no ser asi ; el nunca él comportar de ese...
47567,“Únete a nosotros este jueves 18 de febrero pa...,41,5,9,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47568,“Únete a nosotros este jueves 18 de febrero pa...,41,5,9,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47569,"“‘Hoy tengo que mostrar la Luna’, decía, preoc...",53,4,9,0,52,0,111,7,0,8,51,21,"“ ‘ hoy tener que mostrar el luna ’ , decir , ..."


In [ ]:
df_consolidado=pd.merge(datasetMaterial,df_union,on='dropkey')
df_consolidado

,_id,topic,text,title,year,month,day,key,url,dropkey,Adjs,Advs,Aux,Conj,Det,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma
0,{'$oid': '611d9fe1049812a18b13a8dd'},cultura,el vallenato con guitarra de los hijos de 'la ...,El vallenato con guitarra de los hijos de 'La ...,2020,12,7,cultura-musica-y-libros-vallenatos-de-el-trio-...,http://www.eltiempo.com//cultura/musica-y-libr...,cultura-musica-y-libros-vallenatos-de-el-trio-...,1,0,0,0,3,0,3,0,0,0,0,0,el vallenato con guitarra de el hijo de ' el c...
1,{'$oid': '611d9fe1049812a18b13a8de'},cultura,los secretos de una guru de la comida crudivegana,Los secretos de una gurú de la comida crudivegana,2021,4,10,cultura-gastronomia-entrevista-con-consol-rodr...,http://www.eltiempo.com//cultura/gastronomia/e...,cultura-gastronomia-entrevista-con-consol-rodr...,1,0,0,0,3,0,2,0,0,0,0,0,el secreto de uno guru de el comida crudivegán
2,{'$oid': '611d9fe1049812a18b13a8df'},cultura,obj24062586_13 by yhonatan loaiza on scribdel...,Teatro colombiano que conquista el mundo,2020,9,30,cultura-arte-y-teatro-obras-de-teatro-colombia...,http://www.eltiempo.com//cultura/arte-y-teatro...,OBJ24062586_13 by Yhonatan Loaiza on ScribdEl...,31,8,13,0,42,0,84,6,1,15,67,21,obj24062586_13 by yhonatan loaiza on scribde...
3,{'$oid': '611d9fe1049812a18b13a8e0'},cultura,"""'origen' me regreso a la infancia para record...",Juanes presenta el video de 'Sin medir distancia',2021,7,8,cultura-musica-y-libros-juanes-presenta-el-vid...,http://www.eltiempo.com//cultura/musica-y-libr...,"""'Origen' me regresó a la infancia para record...",7,6,6,0,29,0,42,0,0,12,25,19,""" ' origen ' yo regreso a el infancia para rec..."
4,{'$oid': '611d9fe1049812a18b13a8e5'},cultura,"""a slow fire burning"" es el titulo de la nueva...",Autora de 'La chica del tren' anuncia su nueva...,2020,12,3,cultura-musica-y-libros-la-chica-del-tren-paul...,http://www.eltiempo.com//cultura/musica-y-libr...,"""A Slow Fire Burning"" es el título de la nueva...",21,2,9,0,41,0,62,4,0,18,49,41,""" a slow fire burning "" ser el titulo de el nu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47566,{'$oid': '611d9fe9049812a18b180125'},vida,“el no es asi; el nunca se comporto de esa man...,Consejos para reconocer una relación tóxica y ...,2021,3,4,vida-familia-parejas-como-reconocer-una-relaci...,http://www.eltiempo.com//vida/familia/parejas-...,“Él no es así; él nunca se comportó de esa man...,93,39,53,0,169,0,257,10,0,73,152,140,“ el no ser asi ; el nunca él comportar de ese...
47567,{'$oid': '611d9fe9049812a18b18012b'},vida,“unete a nosotros este jueves 18 de febrero pa...,La Nasa retransmitirá en español el amartizaje...,2021,2,19,vida-ciencia-la-nasa-retransmite-en-espanol-el...,http://www.eltiempo.com//vida/ciencia/la-nasa-...,“Únete a nosotros este jueves 18 de febrero pa...,41,5,9,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47568,{'$oid': '611d9fe9049812a18b18012c'},vida,“unete a nosotros este jueves 18 de febrero pa...,La Nasa retransmitirá en español el amartizaje...,2021,2,19,vida-ciencia-la-nasa-retransmite-en-espanol-el...,http://www.eltiempo.com//vida/ciencia/la-nasa-...,“Únete a nosotros este jueves 18 de febrero pa...,41,5,9,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47569,{'$oid': '611d9fe9049812a18b18012e'},vida,"“‘hoy tengo que mostrar la luna’, decia, preoc...",Cien años sin el genio de Julio Garavito Armero,2020,3,10,vida-ciencia-cien-anos-sin-el-genio-de-julio-g...,http://www.eltiempo.com//vida/ciencia/cien-ano...,"“‘Hoy tengo que mostrar la Luna’, decía, preoc...",53,4,9,0,52,0,111,7,0,8,51,21,"“ ‘ hoy tener que mostrar el luna ’ , decir , ..."


### Explorar

In [ ]:
df.topic.value_counts(normalize=True)

deportes      0.316520
cultura       0.235393
vida          0.139975
politica      0.134090
economia      0.117697
tecnosfera    0.056326
Name: topic, dtype: float64

In [ ]:
def Root(s):
   return [c.lemma_ for c in s if c.dep_ =='ROOT' and c.is_stop ==0 and c.is_punct==0]


In [ ]:
df['textlemma']=df['text'].apply(lambda x: Lemmas(x))
df['Root']=df['text'].apply(lambda x: Root(x))

In [ ]:
df.head(50)

,_id,topic,text,title,year,month,day,key,url,dropkey,POS,Adjs,Advs,Aux,Conj,Det,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma,Root
39570,{'$oid': '611d9fe7049812a18b16e4bd'},tecnosfera,"(la, compania, tecnologica, fundada, por, jeff...",El curioso dispositivo con el cual Amazon busc...,2021,1,7,tecnosfera-novedades-tecnologia-brahms-amazon-...,http://www.eltiempo.com//tecnosfera/novedades-...,La compañía tecnológica fundada por Jeff Bezos...,"[37, 13, 18, 0, 69, 0, 143, 0, 0, 20, 54, 55]",37,13,18,0,69,0,143,0,0,20,54,55,el compania tecnologico fundado por jeff bezos...,"[reconocer, denominar, ‘business, detectar, as..."
41725,{'$oid': '611d9fe8049812a18b174d72'},vida,"(camara, de, descontaminacion, ,, personal, co...",Así es la minuciosa fabricación de la vacuna a...,2021,5,3,vida-ciencia-en-la-planta-de-biontech-asi-es-l...,http://www.eltiempo.com//vida/ciencia/en-la-pl...,"Cámara de descontaminación, personal con ropa ...","[85, 25, 23, 0, 158, 1, 237, 31, 0, 56, 118, 104]",85,25,23,0,158,1,237,31,0,56,118,104,"camara de descontaminacion , personal con ropa...","[funcionar, constatar, explicar, sentir, afirm..."
18106,{'$oid': '611d9fe3049812a18b14e67f'},deportes,"(en, medio, del, fair, play, financiero, ,, qu...",Esta sería la oferta formal del Manchester Cit...,2020,8,27,deportes-futbol-internacional-manchester-citiy...,http://www.eltiempo.com//deportes/futbol-inter...,"En medio del fair play financiero, que obliga ...","[10, 10, 10, 0, 32, 0, 49, 4, 0, 13, 31, 29]",10,10,10,0,32,0,49,4,0,13,31,29,"en medio del fair play financiero , que obliga...","[sonar, levantar, city, preparar, especular]"
35569,{'$oid': '611d9fe6049812a18b1649df'},politica,"(hora, de, ser, sensatos.las, medidas, para, e...",Las reglas de juego para el confinamiento de 1...,2020,3,22,politica-gobierno-las-reglas-de-juego-para-el-...,http://www.eltiempo.com//politica/gobierno/las...,Hora de ser sensatos.Las medidas para evitar e...,"[73, 23, 30, 0, 130, 1, 275, 22, 1, 45, 105, 100]",73,23,30,0,130,1,275,22,1,45,105,100,hora de ser sensatos.las medida para evitar el...,"[autorregulatoria, entender, suceder, adoptar,..."
42140,{'$oid': '611d9fe8049812a18b175a2c'},vida,"(el, 4, de, julio, de, 2012, ,, en, medio, de,...",El físico obsesionado con resolver la fórmula ...,2020,8,24,vida-entrevista-con-john-ellis-uno-de-los-fisi...,http://www.eltiempo.com//vida/entrevista-con-j...,"El 4 de julio de 2012, en medio de un auditori...","[116, 68, 89, 0, 282, 0, 355, 19, 0, 97, 194, ...",116,68,89,0,282,0,355,19,0,97,194,190,"el 4 de julio de 2012 , en medio de uno audito...","[referiar, enfocar, ver, perfectamente.la, tom..."
41785,{'$oid': '611d9fe8049812a18b174f3f'},vida,"(de, acuerdo, con, , la, corporacion, para, l...",Detienen construcción de 20 cabañas en Páramo ...,2021,4,12,vida-medio-ambiente-detienen-construccion-de-2...,http://www.eltiempo.com//vida/medio-ambiente/d...,De acuerdo con la Corporación para la Defensa ...,"[24, 6, 15, 0, 38, 0, 88, 8, 0, 19, 41, 31]",24,6,15,0,38,0,88,8,0,19,41,31,de acuerdo con el corporacion para el defens...,"[iniciar, declarar, desplegar, zona, interesar]"
1886,{'$oid': '611d9fe1049812a18b13cc81'},cultura,"(demi, lovato, es, nuevamente, una, mujer, sol...",Demi Lovato y Max Ehrich cancelan su compromis...,2020,9,25,cultura-entretenimiento-demi-lovato-y-max-ehri...,http://www.eltiempo.com//cultura/entretenimien...,Demi Lovato es nuevamente una mujer soltera. A...,"[27, 21, 25, 0, 57, 0, 74, 5, 0, 38, 63, 50]",27,21,25,0,57,0,74,5,0,38,63,50,demi lovato ser nuevamente uno mujer soltero ....,"[mujer, revelo, interesar, declaro, pareja, sa..."
13154,{'$oid': '611d9fe3049812a18b148c5c'},deportes,"(deportivo, cali, informo, este, viernes, que,...",Deportivo Cali reporta un positivo de covid-19,2020,7,5,deportes-futbol-colombiano-deportivo-cali-repo...,http://www.eltiempo.com//deportes/futbol-colom...,Deportivo Cali informó este viernes que luego ...,"[17, 1, 6, 0, 28, 0, 45, 9, 0, 5, 22, 12]",17,1,6,0,28,0,45,9,0,5,2

In [ ]:
def ner(s):
   return [{c.text:c.label_}for c in s.ents ]

In [ ]:
df['NER']=df['text'].apply(lambda x: ner(x))

In [ ]:
x=list(df['NER'][1]) 
x

NameError: ignored

In [ ]:
y=df['text'].head(1)
y.values

array([la compania tecnologica fundada por jeff bezos, amazon, ya es reconocida en el mercado por la creacion de altavoces inteligentes como alexa y echo, y tambien por aparatos como enchufes inteligentes y microondas compatibles con alexa.ahora, la empresa estaria desarrollando un dispositivo que seria capaz de monitorear el sueno para, entre otras cosas, detectar la apnea del sueno, trastorno que provoca que la respiracion se detenga y vuelva a comenzar repetidamente al dormir.de acuerdo con ‘business insider’, que tuvo acceso a documentos y fuentes cercanas al proyecto, el aparato llevaria el nombre de brahms, que hace referencia a johannes brahms, compositor aleman reconocido por crear la famosa ‘cancion de cuna’. una curiosidad del musico es que se sospecha que padecia apnea del sueno.(tambien puede interesarle: francia multa a google y amazon por no respetar politica de 'cookies').brahms tendria el tamano de la palma de la mano y su diseno permitiria que fuese puesto en la mesita